# Setup

In [1]:
import json
from functools import reduce

from openai import OpenAI
from outlines.fsm.json_schema import build_regex_from_schema, get_schema_from_signature
from pydantic import BaseModel

In [2]:
client = OpenAI(base_url="http://localhost:8000/v1", api_key="-")
client

# Tool To Regex

In [3]:
def regex_or(pattern1, pattern2):
    return f"(?:{pattern1}|{pattern2})"

In [4]:
def sometime_guide(regex_pattern, start_guided_pattern="<tool_call>", end_guided_pattern="</tool_call>"):
    """
    Only do guided generation sometimes, in between start_word and end_word.
    """
    return f".*?(?={start_guided_pattern}){start_guided_pattern}({regex_pattern}).*?(?={end_guided_pattern}){end_guided_pattern}.*"

In [5]:
def is_bfcl(tool):
    return isinstance(tool, dict) and list(tool.keys()) == ['name', 'description', 'parameters']

In [44]:
def tool_to_regex(tool, start_guided_pattern="<tool_call>", end_guided_pattern="</tool_call>", sometimes=False, whitespace_pattern=None, test_category=None):

    if isinstance(tool, list):
        values = [tool_to_regex(_tool, whitespace_pattern=whitespace_pattern, test_category=test_category) for _tool in tool]
        regex_strs = [v[0] for v in values]
        regex_str = reduce(regex_or, regex_strs)
        schema_strs = [v[1] for v in values]
        schema_str = "\n".join(schema_strs)
    elif is_bfcl(tool):
        schema_str = bfcl_function_to_schema(tool, test_category).strip()
        schema_regex = build_regex_from_schema(schema_str, whitespace_pattern)
        regex_str = f'{{"tool_name": "{tool["name"]}", "tool_arguments": {schema_regex}}}'
    elif isinstance(tool, type(BaseModel)):
        schema_json = tool.model_json_schema()
        schema_str = json.dumps(schema_json).strip()
        schema_regex = build_regex_from_schema(schema_str, whitespace_pattern)
        regex_str = f'{{"tool_name": "{schema_json["title"]}", "tool_arguments": {schema_regex}}}'
    elif callable(tool):
        schema_json = get_schema_from_signature(tool)
        schema_str = json.dumps(schema_json).strip()
        schema_regex = build_regex_from_schema(schema_str, whitespace_pattern)
        regex_str = f'{{"tool_name": "{tool.__name__}", "tool_arguments": {schema_regex}}}'
    elif isinstance(tool, str):
        schema_str = tool.replace("\n", " ").replace("  ", " ").strip()
        schema_regex = build_regex_from_schema(schema_str, whitespace_pattern)
        regex_str = f'{{"tool_name": "{json.loads(schema_str)["title"]}", "tool_arguments": {schema_regex}}}'

    if sometimes:
        regex_str = sometime_guide(regex_str)
    # elif not isinstance(tool, list):
    #     regex_str = f"{start_guided_pattern}{regex_str}{end_guided_pattern}"

    return regex_str, schema_str

# Prompt

In [54]:
def get_system_prompt(
    tool_schema,
    tool_list_start="<tool>",
    tool_list_end="</tools>",
    tool_call_start="<tool_call>",
    tool_call_end="</tool_call>",
    tool_response_start="<tool_response>",
    tool_response_end="</tool_response>"
    ):

    system_prompt = """You are a function calling AI model. Your job is to answer the user's questions and you may call one or more functions to do this.


    Please use your own judgment as to whether or not you should call a function. In particular, you must follow these guiding principles:
    1. You may call one or more functions to assist with the user query.
    2. You do not need to call a function. If none of the functions can be used to answer the user's question, please do not make the function call.
    3. Don't make assumptions about what values to plug into functions. If you are missing the parameters to make a function call, please ask the user for the parameters.
    4. You may assume the user has implemented the function themselves.
    5. You may assume the user will call the function on their own. You should NOT ask the user to call the function and let you know the result; they will do this on their own.


    You can only call functions according the following formatting rules:
    Rule 1: All the functions you have access to are contained within {tool_list_start}{tool_list_end} XML tags. You cannot use any functions that are not listed between these tags.

    Rule 2: For each function call return a json object (using quotes) with function name and arguments within {tool_call_start}\n{{ }}\n{tool_call_end} XML tags as follows:
    * With arguments:
    {tool_call_start}\n{{"tool_name": "function_name", "tool_arguments": {{"argument_1_name": "value", "argument_2_name": "value"}} }}\n{tool_call_end}
    * Without arguments:
    {tool_call_start}\n{{ "tool_name": "function_name", "tool_arguments": {{}} }}\n{tool_call_end}
    In between {tool_call_start} and{tool_call_end} tags, you MUST respond in a valid JSON schema.
    In between the {tool_call_start} and {tool_call_end} tags you MUST only write in json; no other text is allowed.

    Rule 3: If user decides to run the function, they will output the result of the function call between the {tool_response_start} and {tool_response_start} tags. If it answers the user's question, you should incorporate the output of the function in your answer.


    Here are the tools available to you:
    {tool_list_start}\n{tool_schema}\n{tool_list_end}

    Remember, don't make assumptions about what values to plug into functions. If you are missing the parameters to make a function call, please ask the user for the parameters. Do not be afraid to ask.
    """

    return system_prompt.format(
        tool_list_start=tool_list_start,
        tool_list_end=tool_list_end,
        tool_call_start=tool_call_start,
        tool_call_end=tool_call_end,
        tool_response_start=tool_response_start,
        tool_response_end=tool_response_end,
        tool_schema=tool_schema,
        )

# Parse Output

In [46]:
def is_tool(text):
    return "<tool_call>" in text and "</tool_call>" in text

In [47]:
def parse_tool(text, start_tag = "<tool_call>", end_tag = "</tool_call>"):

        start_index = text.find(start_tag)
        if start_index == -1:
            return None

        end_index = text.find(end_tag, start_index + len(start_tag))
        if end_index == -1:
            # if end_tag not present, check for last occurrence of start_tag and use this as the end_index
            end_index = text.rfind(start_tag, start_index + len(start_tag))
            if end_index == -1:
                return None

        function_call = text[start_index + len(start_tag):end_index].strip(" \n")
        return function_call

# Run it

In [51]:
def send_email(sender: str, recipient: str, message: str):
    return "Hi"

class User(BaseModel):
    name: str
    last_name: str
    id: int

schema = """
{
  "title": "User",
  "type": "object",
  "properties": {
    "name": {"type": "string"},
    "last_name": {"type": "string"},
    "id": {"type": "integer"}
  }
}
"""

# tools = [send_email, User, schema]
tools = send_email
regex_str, tool_schema = tool_to_regex(tools)

start_guided_pattern = "<tool_call>"
end_guided_pattern = "</tool_call>"
regex_str = f"{start_guided_pattern}{regex_str}{end_guided_pattern}"

print(regex_str)
print(tool_schema)

<tool_call>{"tool_name": "send_email", "tool_arguments": \{[\n ]*"sender"[\n ]*:[\n ]*"(?:[^"\\\x00-\x1f\x7f-\x9f]|\\.)*"[\n ]*,[\n ]*"recipient"[\n ]*:[\n ]*"(?:[^"\\\x00-\x1f\x7f-\x9f]|\\.)*"[\n ]*,[\n ]*"message"[\n ]*:[\n ]*"(?:[^"\\\x00-\x1f\x7f-\x9f]|\\.)*"[\n ]*\}}</tool_call>
{"properties": {"sender": {"title": "Sender", "type": "string"}, "recipient": {"title": "Recipient", "type": "string"}, "message": {"title": "Message", "type": "string"}}, "required": ["sender", "recipient", "message"], "title": "Arguments", "type": "object"}


In [52]:
user_query = "Can you send an email from Alice (alice@gmail.com) to Bob  (bob@databricks.com) saying 'We cracked the code!'?"
system_prompt = get_system_prompt(tool_schema)

messages = [
  {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_query}
  ]
print(messages)

[{'role': 'system', 'content': 'You are a function calling AI model. Your job is to answer the user\'s questions and you may call one or more functions to do this.\n\n\n    Please use your own judgment as to whether or not you should call a function. In particular, you must follow these guiding principles:\n    1. You may call one or more functions to assist with the user query.\n    2. You do not need to call a function. If none of the functions can be used to answer the user\'s question, please do not make the function call.\n    3. Don\'t make assumptions about what values to plug into functions. If you are missing the parameters to make a function call, please ask the user for the parameters.\n    4. You may assume the user has implemented the function themselves.\n    5. You may assume the user will call the function on their own. You should NOT ask the user to call the function and let you know the result; they will do this on their own.\n\n\n    You can only call functions accor

In [53]:
completion = client.chat.completions.create(
  model="databricks/dbrx-instruct",
  max_tokens=250,
  messages=messages,
  extra_body=dict(guided_regex=regex_str, guided_decoding_backend="outlines"),
  )

raw_text = completion.choices[0].message.content
print(raw_text)

InternalServerError: Internal Server Error

In [24]:
tool = None
if is_tool(raw_text):
    tool = parse_tool(raw_text)
print(tool)

{"name": "send_email", "arguments": {"sender": "alice@gmail.com", "recipient": "bob@databricks.com", "message": "We cracked the code!"} }
